### Let's deploy an application that explores rate limiting

In [ ]:
# get to the right directory
cd ~/projects/spring-cloud-gateway-workshop/spring-boot-version

In [ ]:
# run the app locally
mvn -DskipTests spring-boot:run

In [ ]:
# build the app
mvn -DskipTests clean install

In [4]:
#see which apps are currently deployed
cf apps

Getting apps in org scg-workshop / space demo as brian...
OK

name                     requested state   instances   memory   disk   urls
accounts                 started           1/1         1G       1G     accounts-optimistic-cassowary-xj.cfapps.pcf.cloud
animal-rescue-backend    started           1/1         1G       1G     cs-animal-rescue-backend.apps.internal
animal-rescue-frontend   started           1/1         1G       1G     cs-animal-rescue-frontend.apps.internal
custom-gateway           started           1/1         1G       1G     custom-gateway.cfapps.pcf.cloud
portfolio                started           1/1         1G       1G     portfolio-kind-badger-iy.cfapps.pcf.cloud
quotes                   started           1/1         1G       1G     quotes-responsive-zebra-bv.cfapps.pcf.cloud
ratelimiting             started           1/1         128M     1G     ratelimiting.cfapps.pcf.cloud, ratelimiting.apps.internal
testgateway              started           1/1         1G   

In [ ]:
#create a new service instance of the gateway
cf push

In [6]:
#test out redis rate limiting
# https://custom-gateway.cfapps.pcf.cloud/ratelimit/bucket
for i in {1..20}; do sleep .1; curl --dump-header - https://custom-gateway.cfapps.pcf.cloud/ratelimit/bucket?user=1 ;echo '\n';done


HTTP/1.1 404 Not Found
Cache-Control: no-cache, no-store
Content-Type: text/plain; charset=utf-8
X-Cf-Routererror: unknown_route
X-Content-Type-Options: nosniff
Date: Thu, 06 Aug 2020 20:09:22 GMT
Content-Length: 83

404 Not Found: Requested route ('custom-gateway.cfapps.pcf.cloud') does not exist.
\n
HTTP/1.1 404 Not Found
Cache-Control: no-cache, no-store
Content-Type: text/plain; charset=utf-8
X-Cf-Routererror: unknown_route
X-Content-Type-Options: nosniff
Date: Thu, 06 Aug 2020 20:09:22 GMT
Content-Length: 83

404 Not Found: Requested route ('custom-gateway.cfapps.pcf.cloud') does not exist.
\n
HTTP/1.1 404 Not Found
Cache-Control: no-cache, no-store
Content-Type: text/plain; charset=utf-8
X-Cf-Routererror: unknown_route
X-Content-Type-Options: nosniff
Date: Thu, 06 Aug 2020 20:09:22 GMT
Content-Length: 83

404 Not Found: Requested route ('custom-gateway.cfapps.pcf.cloud') does not exist.
\n
HTTP/1.1 404 Not Found
Cache-Control: no-cache, no-store
Content-Type: text/plain; charset=

In [ ]:
# test out 1 per second by calling 4 times per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec
# should get a successful response 1 in 4 times, may come across as 1 in 3 due to timing
for i in {1..20}; do sleep .25; curl -o -I -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec ;echo '\n';done

In [ ]:
#test out 1 per second by calling 1 time per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec
# should be all 200s
for i in {1..10}; do sleep 1; curl -o -I -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/1persec ;echo '\n';done

In [ ]:
#test out 10 per minute by calling 1 time per second
# https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/10permin
for i in {1..20}; do sleep 1; curl -o -I -L -s -w "%{http_code}" https://ratelimit-gateway.cfapps.pcf.cloud/ratelimiting/10permin ;echo '\n';done